<center> <h1>Market Data</h1> </center>
<img src="img/both.png" width=600 /> 

In this notebook, we pull financial data that gives an overview of the state of the different markets the insured companies are making business on

More precisely, we pull the state of market indices from two sources :
some directly from scraping yahoo finance through the yfinance library, and some from parsing index data downloaded as excel sheets from the s&p Dow Jones website (www.spglobal.com)

Since we did not always find data that reflected the state of the world economy, some of it have been replaced by sectorial data of the american economy

**output:**

We output a 2D table, which to a year, and a sector gives a *financial ratio* (or *growth ratio*)
which is calculated as such

$$ \frac{\text{mean value of the index on the current year}}{\text{mean value of the index on the previous year}}$$

(which is a good indicator of wether the market is doing well)

The corresponding value is then added to each contract in the preprocessing notebook

## industry indexes

In [69]:
import re
import os

import numpy as np
import pandas as pd

import yfinance as yf

In [70]:
EXCEL_DIR = "./data/market-indexes/" #directory where data from s&p500 resides

### List different industries

In [71]:
df = pd.read_excel('./data/DATASET_Train2.xlsx')
#df["SUBSECTOR"].unique()
categories = df["MAIN_PRICING_CATEG"].unique()
categories = np.append(categories, "Railway manufacturing industry")
categories

/home/tris/.local/lib/python3.9/site-packages/openpyxl/styles/stylesheet.py:214: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


array(['Mining - Hard Rock', 'Mining - Bulk commodities',
       'Mining - Underground coal', 'Chemicals', 'Metallurgy',
       'Mining - Other', 'Building materials', 'Petrochemical',
       'Crude Processing - Refining', 'Gas Processing',
       'Midstream - Transportation/Storage', 'Specialty chemicals',
       'Warehouse', 'Other Onshore', 'Other categories',
       'Rubber & Plastic Manufacturing', 'Mechanical industries', 'Power',
       'Commercial buildings', 'Food producers', 'Water Treatment',
       'High Tech Industries', 'Infrastructure', 'Metal Works',
       'Pulp & paper', 'Textile industry', 'Transportation & Logistics',
       'Waste Treatment', 'Wood industry', 'Beverage producers',
       'Leisure industry', 'Pharmaceuticals & fine chemicals',
       'Public buildings', 'Aerospace & Defence',
       'Telecommunications & Media', 'Printing industry',
       'Automotive & Parts', 'Glass/ Ceramic/Others', 'Tobacco',
       'Railway manufacturing industry'], dtype=objec

functions for scraping data (either from excels downloaded from the s&p500 websites, or directly from yahoo finance)

In [72]:
def read_sp_excel(filename):
    filename = os.path.join(EXCEL_DIR, filename)
    excel_data = pd.read_excel(filename, skiprows=6, skipfooter=1)
    columns = excel_data.columns
    excel_data.set_index(columns[0], inplace=True)
    excel_data.rename(columns={columns[1]:"value"}, inplace=True)
    excel_data.dropna(inplace=True)
    excel_data.sort_index(inplace=True)
    return excel_data

def from_yahoo(symbol):
    index = yf.Ticker(symbol)
    index_data = index.history(interval="1mo", period="max")
    index_data.rename(columns = {"Close" : "value"}, inplace=True)
    index_data.dropna(inplace=True)
    index_data.sort_index(inplace=True)
    return index_data["value"]
    
metalurgy_index = from_yahoo("STEEL")
metalurgy_index

mining_index = read_sp_excel("mining.xls")
mining_index

,value
Effective date,
2011-01-31,120.84
2011-02-01,123.68
2011-02-02,123.87
2011-02-03,126.01
2011-02-04,125.04
...,...
2021-02-01,104.81
2021-02-02,102.26
2021-02-03,102.53


In [47]:
#mining
mining_index = read_sp_excel("mining.xls")

#Metalurgy
metalurgy_index = from_yahoo("STEEL")


#Building materials
materials_index = read_sp_excel("materials.xls") #S&P materials


#Petrochemical & Crude Processing - refining & rubber / plastic
oil_index = from_yahoo("BZ=F") #brent barrel


# Gas
gas_index = read_sp_excel("gas.xls")


# chemicals
chemicals_index = from_yahoo("^SP500-151010")


# Transportation
transportation_index = from_yahoo("^SPSITNTR")


# Energy
energy_index = from_yahoo("^GSPE")


# Real estate
realestate_index = from_yahoo("^DJUSRE")


# Food and drink 
fd_index = from_yahoo("^SPSIFB")
fd_index = read_sp_excel("food-drink.xls")



# wood
wood_index = from_yahoo("WOOD")


# aerospace & defense
aero_index = from_yahoo("XAR")


# textile, apparel & luxury goods
#tex_index = from_yahoo("^SP500-252030")
tex_index = read_sp_excel("discretionary.xls")


#industry
indus_index = from_yahoo("^SP500-20")


#printing
print_index = from_yahoo("^SP500-25401040")


#water 
water_index = from_yahoo("CGW")


#high tech 
ht_index = from_yahoo("^IXIC")


#infrastructure
infra_index = from_yahoo("^SPGTINNTE")


# pharmaceuticals 
pharma_index = from_yahoo("XPH")


#telecom
telecom_index = from_yahoo("^SPSITE")


# automotive & components
auto_index = from_yahoo("^SP500-2510TR")


#general index 
general_index = from_yahoo("^GSPC") # S&P500, it is used for missing values, or industry we do not have data on

data = general_index
indus_index = general_index
tex_index

,value
Effective date,
2020-02-18,1043.39
2020-02-19,1048.28
2020-02-20,1048.16
2020-02-21,1031.02
2020-02-24,994.64
...,...
2021-02-09,1383.37
2021-02-10,1370.33
2021-02-11,1366.17


In [62]:
#data.loc["1985-02-01"].Close

def mean(data, year):
    #return sum(data.loc["{}-{}-01".format(year, month)].Close for month in range(1, 13)) / 12
    r =  data.loc["{}-01-01".format(year): "{}-12-31".format(year)].mean()
    if isinstance(r, float):
        return r
    else:
        return r.astype(float).iloc[0]
def datetime_of_year(year):
    return pd.to_datetime("{}-01-01".format(year))

mean(tex_index, 2020)
tex_index
tex_index["2019-01-01": "2020-02-28"]
#tex_index.loc[tex_index["Effective date"]]

,value
Effective date,
2020-02-18,1043.39
2020-02-19,1048.28
2020-02-20,1048.16
2020-02-21,1031.02
2020-02-24,994.64
2020-02-25,968.73
2020-02-26,962.01
2020-02-27,922.88
2020-02-28,915.77


In [63]:
#df["market"] = 0.

output = pd.DataFrame(columns=["MAIN_PRICING_CATEG" , *range(2008, 2019)])

for categ in categories :
    market = None
    if re.search("Mining -.*", categ):
        market = mining_index
    elif categ in ("Metallurgy", 'Metal Works'):
        market = metalurgy_index
    elif categ in ("Building materials",'Glass/ Ceramic/Others',):
        market = materials_index
    elif categ in ("Petrochemical",  'Crude Processing - Refining', 'Other Onshore' ):
        market = oil_index
    elif categ in ('Gas Processing',):
        market = gas_index
    elif categ in ('Midstream - Transportation/Storage', "Warehouse", 'Transportation & Logistics',):
        market = transportation_index
    elif categ in ("Chemicals", 'Specialty chemicals',):
        market = chemicals_index
    #elif categ in ('Rubber & Plastic Manufacturing',):
    #    market = plastic_index
    elif categ in ('Mechanical industries',):
        market = indus_index
    elif categ in ('Power',):
        market = energy_index
    elif categ in ('Commercial buildings','Public buildings',):
        market = realestate_index
    elif categ in ('Food producers', 'Beverage producers','Tobacco'):
        market = fd_index
    elif categ in ('Water Treatment',):
        market = water_index
    elif categ in ('High Tech Industries',):
        market = ht_index
    elif categ in ('Infrastructure',):
        market = infra_index
    #elif categ in ('Pulp & paper',):
    #    market = infra_index
    elif categ in ('Textile industry',):
        market = tex_index
    #elif categ in ('Waste Treatment',):
        #market = tex_index
    elif categ in ('Wood industry',):
        market = wood_index
    #elif categ in ('Leisure industry',):
    #    market = wood_index
    elif categ in ('Pharmaceuticals & fine chemicals',):
        market = pharma_index
    elif categ in ('Aerospace & Defence',):
        market = aero_index
    elif categ in ('Telecommunications & Media', ):
        market = telecom_index
    elif categ in ('Printing industry',):
        market = print_index
    elif categ in ('Automotive & Parts',):
        market = auto_index 
    else : 
        market = general_index
    data = market
    min_index = data.index.min()
    line = {year : (mean(data, year) / mean(data, year-1) if datetime_of_year(year - 1) >= min_index else None ) for year in range(2008, 2021)}
    line["MAIN_PRICING_CATEG"] = categ
    output = output.append(line, ignore_index=True)

In [64]:
output #lots of missing values for older dates

,MAIN_PRICING_CATEG,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,Mining - Hard Rock,NaN,NaN,NaN,NaN,NaN,0.823210,1.015485,0.772761,0.995933,1.159407,1.042942,1.076494,1.177278
1,Mining - Bulk commodities,NaN,NaN,NaN,NaN,NaN,0.823210,1.015485,0.772761,0.995933,1.159407,1.042942,1.076494,1.177278
2,Mining - Underground coal,NaN,NaN,NaN,NaN,NaN,0.823210,1.015485,0.772761,0.995933,1.159407,1.042942,1.076494,1.177278
3,Chemicals,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Metallurgy,1.083452,0.596295,1.447206,1.011215,0.756427,0.938528,1.015180,0.623003,1.050668,1.407641,1.102580,0.852096,0.771530
5,Mining - Other,NaN,NaN,NaN,NaN,NaN,0.823210,1.015485,0.772761,0.995933,1.159407,1.042942,1.076494,1.177278
6,Building materials,NaN,NaN,NaN,NaN,NaN,0.759859,0.959137,0.839084,1.103117,1.068338,0.968378,1.048017,1.226500
7,Petrochemical,NaN,0.657843,1.254257,1.393116,0.993469,0.972135,0.899338,0.557802,0.844483,1.212092,1.284290,0.906100,0.641627
8,Crude Processing - Refining,NaN,0.657843,1.254257,1.393116,0.993469,0.972135,0.899338,0.557802,0.844483,1.212092,1.284290,0.906100,0.641627
9,Gas Processing,NaN,NaN,NaN,NaN,NaN,1.300278,1.129946,0.626835,0.979067,1.173022,1.001111,0.829014,0.874885


In [65]:
default = output["MAIN_PRICING_CATEG"].str.find("Mechanical industries")
"""for column_n, column_name in enumerate(output.columns[1:]):
    for i in range(len(output)):
        if output[column][i] is None:
            output[column][i] = output[column]"""
#print(output["MAIN_PRICING_CATEG"].str.find("Mechanical industries"))
for column_n, column in enumerate(output.columns[1:]):
    for i in range(len(output)):
        #print(output[i])
        if output[column][i] is None or output[column][i] != output[column][i]  :
            output[column][i] = pd.to_numeric(output[column][output["MAIN_PRICING_CATEG"] == "Mechanical industries"]).iloc[0]




<ipython-input-65-437474a9cbf0>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  output[column][i] = pd.to_numeric(output[column][output["MAIN_PRICING_CATEG"] == "Mechanical industries"]).iloc[0]


In [66]:
pd.to_numeric(output[2008], errors="coerce")
output

,MAIN_PRICING_CATEG,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,Mining - Hard Rock,0.822154,0.780530,1.192051,1.132729,1.082568,0.823210,1.015485,0.772761,0.995933,1.159407,1.042942,1.076494,1.177278
1,Mining - Bulk commodities,0.822154,0.780530,1.192051,1.132729,1.082568,0.823210,1.015485,0.772761,0.995933,1.159407,1.042942,1.076494,1.177278
2,Mining - Underground coal,0.822154,0.780530,1.192051,1.132729,1.082568,0.823210,1.015485,0.772761,0.995933,1.159407,1.042942,1.076494,1.177278
3,Chemicals,0.822154,0.780530,1.192051,1.132729,1.082568,1.191697,1.176797,1.055294,1.026267,1.170656,1.110821,1.072877,1.094961
4,Metallurgy,1.083452,0.596295,1.447206,1.011215,0.756427,0.938528,1.015180,0.623003,1.050668,1.407641,1.102580,0.852096,0.771530
5,Mining - Other,0.822154,0.780530,1.192051,1.132729,1.082568,0.823210,1.015485,0.772761,0.995933,1.159407,1.042942,1.076494,1.177278
6,Building materials,0.822154,0.780530,1.192051,1.132729,1.082568,0.759859,0.959137,0.839084,1.103117,1.068338,0.968378,1.048017,1.226500
7,Petrochemical,0.822154,0.657843,1.254257,1.393116,0.993469,0.972135,0.899338,0.557802,0.844483,1.212092,1.284290,0.906100,0.641627
8,Crude Processing - Refining,0.822154,0.657843,1.254257,1.393116,0.993469,0.972135,0.899338,0.557802,0.844483,1.212092,1.284290,0.906100,0.641627
9,Gas Processing,0.822154,0.780530,1.192051,1.132729,1.082568,1.300278,1.129946,0.626835,0.979067,1.173022,1.001111,0.829014,0.874885


In [68]:
output.to_csv("./data/market_values.csv")